In [78]:
from llama_index.embeddings.nomic import NomicEmbedding
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    Document,
    get_response_synthesizer,
)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from typing import Annotated
import os

In [79]:
documents = SimpleDirectoryReader("./data/test/").load_data()
nomic_api_key = os.environ.get("NOMIC_API_KEY")
embed_model = NomicEmbedding(
    api_key=nomic_api_key,
    model_name="nomic-embed-text-v1",
    task_type="search_document"
)
service_context = ServiceContext.from_defaults(
    embed_model=embed_model, chunk_size=1024,
)
index = VectorStoreIndex.from_documents(
    documents=documents, service_context=service_context, show_progress=True
)


/var/folders/wv/b_h63k2j0997cxn743g60jn40000gn/T/ipykernel_38252/908314802.py:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [77]:
embed_model = NomicEmbedding(
    api_key=nomic_api_key,
    model_name="nomic-embed-text-v1",
    task_type="search_query"
)

service_context = ServiceContext.from_defaults(
    embed_model=embed_model
)

index = VectorStoreIndex.from_documents(documents, embedding_model=embed_model)
#search_query_retriever = index.as_retriever(service_context=service_context, similarity_top_k=2)

/var/folders/wv/b_h63k2j0997cxn743g60jn40000gn/T/ipykernel_38252/3146419878.py:7: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [82]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

In [83]:
response = query_engine.query("What is the effectiveness of alternative medicine?")
print(response)

The effectiveness of alternative medicine was analyzed in a study involving 150 inflammatory bowel disease (IBD) patients. The study found that 60% of the patients used alternative and complementary therapies (ACM), with 31% using exercise, diet, and prayer exclusively. Patients reported subjective benefits from these therapies, with 95% feeling helped by trials of exercise, diet, and prayer, compared to 67% feeling helped by other ACM. Additionally, ACM was sought for various symptoms such as pain/cramps, diarrhea, and gas/bloating, with diet being the most commonly used therapy.


# Mistral Eval + RAG

In [24]:
import json
import os
import matplotlib.pyplot as plt
import glob
from openai import OpenAI
import requests
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

In [25]:
from dotenv import load_dotenv
load_dotenv()
open_api_key = os.getenv("OPENAI_API_KEY")

In [28]:
def prompt(payload):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {open_api_key}"
    }

    try:
        resp = requests.post("https://api.openai.com/v1/chat/completions", 
                             headers=headers, json=payload)
        data = resp.json()
        resp_dict = data['choices'][0]['message']['content']

        return resp_dict
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"An error occurred: {err}")
    except KeyError as key_err:
        print(f"Key error: {key_err} - the structure of the response might not be as expected.")
    
    return None

In [32]:
# load the data as current dir (one level up) + data folder + file 
data = json.load(open('../pubmedqa/data/ori_pqal.json'))
labels = json.load(open('../pubmedqa/data/test_ground_truth.json'))
len(data), len(labels)

(1000, 500)

In [42]:
predictions, gt = {}, {}

class_dict = { 'no': 0, 'yes': 1, 'maybe': 2}

for idx, question in data.items():
    # Retrieve the most relevant nodes for the current question
    print(f"Retrieving nodes for question: {question["QUESTION"]} ...")
    retrieved_nodes_nomic = retriever_nomic.retrieve(question["QUESTION"])
    print(f"Retrieved nodes: {retrieved_nodes_nomic}")
    # Combine the texts of the retrieved nodes to form the context
    combined_context = " ".join([n['text'] for n in retrieved_nodes_nomic])

    # Create the prompt using the combined context and the current question
    text_prompt = f"Given the context: {combined_context} . Answer the question: {question} . The answer can only be one word and it should be either 'yes', 'no', or 'maybe'."

    prompt_data = {
        "model": 'gpt-4-turbo-2024-04-09', 
        "temperature": 0.1,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": text_prompt},
                ],
            }
        ],
        "max_tokens": f"{30}",
    }
    resp = prompt(prompt_data)
    
    # Update predictions and ground truth mappings
    predictions.update({idx: class_dict[resp.lower()]})
    gt.update({idx: class_dict[labels[idx]]})


Retrieving nodes for question: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death? ...
Retrieved nodes: [NodeWithScore(node=TextNode(id_='c76b30f6-3a7e-4efd-9539-9b12d2a1969a', embedding=None, metadata={'file_path': '/Users/jcllobet/Code/Github/pubmedqa-benchmark/data/test/qapairs_pqau_split_part_1.json', 'file_name': 'qapairs_pqau_split_part_1.json', 'file_type': 'application/json', 'file_size': 21285280, 'creation_date': '2024-04-11', 'last_modified_date': '2024-04-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f971a2fe-32a7-4dac-9d81-cd8550c7a27d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/jcllobet/Code/Github/pubmedqa-benchmark/dat

TypeError: 'NodeWithScore' object is not subscriptable

In [ ]:
len(predictions), len(gt)

In [ ]:
accuracy_score(list(gt.values()), list(predictions.values() ))

In [ ]:
f1_score(list(gt.values()), list(predictions.values()), average='macro')